In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
JupyterDash.infer_jupyter_proxy_config()

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# from animal_shelter import AnimalShelter
from CRUD_Python_Module import AnimalShelter  # <<< CHANGED


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

# Kept for compatibility with the starter text; your Project One module uses .env
username = "aacuser"
password = "SNHU1234"

# shelter = AnimalShelter(username, password)
shelter = AnimalShelter()  # uses your .env (URI/DB/Collection) from Project One  <<< CHANGED


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

# --- Flexible read handling (supports several return shapes) ---  <<< CHANGED
try:
    _res = shelter.read({}, {"_id": 0})  # prefer projection so _id doesn't reach Dash
except TypeError:
    _res = shelter.read({})

# Normalize to list[dict] -> records
records = None
if isinstance(_res, list):
    records = _res
elif hasattr(_res, "data"):
    records = getattr(_res, "data")
elif isinstance(_res, dict):
    records = _res.get("data", _res.get("result", []))
elif isinstance(_res, tuple) and len(_res) >= 2:
    records = _res[1]
else:
    try:
        records = list(_res)
    except Exception:
        records = []

df = pd.DataFrame.from_records(records)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True, errors='ignore')  # safe even if already excluded

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div("Unique ID: Erek Mackroy", style={'margin':'8px 0', 'fontWeight':'bold'}),  # <<< ADDED for screenshot
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        page_action='native',
        page_current=0,
        page_size=10,
        row_selectable='single',
        selected_rows=[0],  # select first row so map renders on load
        style_table={'height': '420px', 'overflowY': 'auto', 'minWidth': '100%'},
        style_cell={'textAlign': 'left', 'minWidth': '120px', 'width': '120px',
                    'maxWidth': '240px', 'whiteSpace': 'normal'},
    ),
    html.Br(),
    html.Hr(),
    html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # FIXME Add in the code for your geolocation chart
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data to display.")]
    
    dff = pd.DataFrame(viewData)

    # pick selected row or default to first
    row = (index or [0])[0]
    row = max(0, min(row, len(dff) - 1))

    # try common latitude/longitude column names
    lat_col = next((c for c in dff.columns if c.lower() in ["location_lat", "latitude", "lat"]), None)
    lon_col = next((c for c in dff.columns if c.lower() in ["location_long", "longitude", "lon", "lng"]), None)

    lat = lon = None
    if lat_col and lon_col:
        try:
            lat = float(dff.loc[row, lat_col])
            lon = float(dff.loc[row, lon_col])
        except Exception:
            lat = lon = None

    # fallback to GeoJSON-style location.coordinates [lon, lat]
    if (lat is None or lon is None) and 'location' in dff.columns:
        try:
            loc = dff.loc[row, 'location']
            coords = loc.get('coordinates') if isinstance(loc, dict) else None
            if coords and len(coords) == 2:
                lon = float(coords[0]); lat = float(coords[1])
        except Exception:
            lat = lon = None

    if lat is None or lon is None:
        return [html.Div("Selected row does not have latitude and longitude.")]

    name = dff.loc[row, 'name'] if 'name' in dff.columns else "Unknown"
    breed = dff.loc[row, 'breed'] if 'breed' in dff.columns else "Animal"

    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon],
            zoom=11,
            children=[
                dl.TileLayer(id='base-layer-id'),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H4(str(name)),
                            html.P(f"lat: {lat:.5f}, lon: {lon:.5f}")
                        ])
                    ]
                )
            ]
        )
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
# app.run_server()
app.run_server(mode='inline', debug=False)  # add port=8051 if the default port is busy



 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_alive_38612824-45d7-459d-be4b-96c31cedb2d7 HTTP/1.1" 200 -


127.0.0.1 - - [10/Oct/2025 13:39:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1752168217.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1752168217.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1752168217.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_8m1752168217.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2025 13:39:21] "GET /_dash-component-suites/dash_leaflet/dash_leaflet.v0_1_23m1752168237.min.js HTTP/1.1